In [17]:
!ollama list

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


NAME                       ID              SIZE      MODIFIED     
deepseek-r1:32b            38056bbcbb2d    19 GB     13 days ago     
qwen2.5-coder:32b          4bd6cbf2d094    19 GB     2 weeks ago     
llama3.2:3b                a80c4f17acd5    2.0 GB    3 weeks ago     
mistral:latest             f974a74358d6    4.1 GB    4 weeks ago     
deepseek-r1:14b            ea35dfe18182    9.0 GB    4 weeks ago     
llama3.1:latest            46e0c10c039e    4.9 GB    5 weeks ago     
nomic-embed-text:latest    0a109f422b47    274 MB    6 weeks ago     
llama3.2-vision:latest     085a1fdae525    7.9 GB    6 weeks ago     
llama3:latest              365c0bd3c000    4.7 GB    7 weeks ago     
llama2:latest              78e26419b446    3.8 GB    2 months ago    


In [18]:
from qdrant_client.http.models import HnswConfig, HnswConfigDiff
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable.passthrough import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
import uuid

# Initial variables
file = "/home/namnh1/rag-llm-chatbot/Quản lý mô hình AI camera.pdf"
chunk_size = 500
chunk_overlap = 100
collection_name = "ai-camera-deepseek"

# Initialize model
llm = ChatOllama(
        model="deepseek-r1:14b",
        temperature=0,
        streaming=True
    )
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")
loader = PyMuPDFLoader(file)
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", "(?<=\. )", " ", ""]
        )
docs = splitter.split_documents(documents)

In [19]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue
from rank_bm25 import BM25Okapi

In [20]:
from sentence_transformers import CrossEncoder

class Qdrant:
    def __init__(self, host = "10.100.140.54", port = 6333):
        self.client = QdrantClient(host= host, port= port)
        self.reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

    def create_collection(self, collection_name):
        existing_collections = [col.name for col in self.client.get_collections().collections]
        if collection_name not in existing_collections:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(
                    size=len(embeddings.embed_query("sample text")), 
                    distance=Distance.COSINE
                ),
                hnsw_config=HnswConfigDiff(
                    m=16,                    # Number of connections per node
                    ef_construct=100,       # Higher value = better recall
                    full_scan_threshold=10000  # Adding the missing required field
                )
            )


    def insert_documents(self, collection_name: str, doc_chunks: List, embeddings):
        vectors = [
            embeddings.embed_query(chunk.page_content) for chunk in doc_chunks
        ]
        payloads = [
            {"id": str(uuid.uuid4()), "text": chunk.page_content} for chunk in doc_chunks
        ]
        
        points = [
            PointStruct(id=payload["id"], vector=vectors[i], payload=payload) for i, payload in enumerate(payloads)
        ]

        # Sử dụng upsert thay vì upload_collection
        self.client.upsert(
            collection_name=collection_name,
            points=points
        )


    def search_database(self, collection_name, question, embeddings, limit=3):
        query_vector = embeddings.embed_query(question)
        search_result = self.client.query_points(
            collection_name=collection_name,
            query=query_vector,
            limit=limit
        ).points
        if not search_result:
            return "Không tìm thấy kết quả."
        try:
            text_result = "\n\n".join(hit.payload['text'] for hit in search_result)
            return text_result
        except KeyError as e:
            return "Lỗi: Cấu trúc payload không hợp lệ."
        except Exception as e:
            return "Lỗi: Không thể xử lý kết quả tìm kiếm."

    def rerank_documents(self, question: str, documents: List[str], top_k: int = 3) -> List[str]:
        """
        Rerank a list of documents based on relevance to the question using CrossEncoder.
        """
        if not documents:
            return []
        
        rerank_inputs = [[question, doc] for doc in documents]
        scores = self.reranker.predict(rerank_inputs, batch_size=32)
        sorted_pairs = sorted(zip(scores, documents), reverse=True)
        return [doc for _, doc in sorted_pairs][:top_k]
    
    def search_database_fusion_bm250(self, collection_name, question, embeddings, doc_chunks, limit=5):
        # 🔍 BM25 Search
        texts = [doc.page_content for doc in doc_chunks]  # Lấy toàn bộ văn bản đã index
        tokenized_corpus = [text.split() for text in texts]
        bm25 = BM25Okapi(tokenized_corpus)
        bm25_scores = bm25.get_scores(question.split())
    
        top_bm25_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:limit]
        top_bm25_results = [texts[i] for i in top_bm25_indices]
    
        # 🔍 Vector Search
        query_vector = embeddings.embed_query(question)
        search_result = self.client.search(
            collection_name=collection_name,
            query_vector=query_vector,
            limit=limit
        )
    
        # 🌀 Fusion Search (Kết hợp BM25 + Vector Search)
        fusion_results = top_bm25_results + [hit.payload['text'] for hit in search_result if 'text' in hit.payload]
        fusion_results = list(set(fusion_results))  # Loại bỏ kết quả trùng lặp

        # Rerank combined results
        reranked_docs = self.rerank_documents(question, fusion_results, top_k=limit)
        
        return "\n\n".join(fusion_results[:limit])


In [21]:
prompt_template = """
Bạn là một trợ lý AI thông minh. Tên bạn là DetNam. Bạn có quyền truy cập vào tài liệu/tri thức được cung cấp từ nguồn bên ngoài. Dựa trên thông tin đó, hãy trả lời theo cách chính xác, tự nhiên và hữu ích nhất.
Conversation history:
{history}

Trả lời câu hỏi: {question}  
- Dựa trên thông tin từ nguồn dữ liệu, tổng hợp và trình bày câu trả lời. Nếu không đủ thông tin, báo rõ và bổ sung kiến thức chung nếu phù hợp.
- Nếu câu hỏi không nằm trong tài liệu và không liên quan tới tài liệu, hãy sử dụng kiến thức bên ngoài để trả lời.
Retrieved context:
{context}

[Details]  
- Sử dụng ngôn ngữ tự nhiên, ngắn gọn.  
- Định dạng: [Tóm tắt] - [Giải thích]- [Ví dụ mẫu]- [Flow chart] - [Kết luận].
- Đưa thêm gợi ý câu hỏi tiếp theo

[Example]
- Xin chào/hi/hello -> tôi tên DetNam, trợ lý thông minh, tôi có thể giúp gì cho bạn đây.
"""

In [22]:
def build_chain(llm, qdrant_client, collection_name, embeddings, question, history):
    retrieved_context = qdrant_client.search_database_fusion_bm250(collection_name, question, embeddings, docs)
    
    pipeline = (
        RunnablePassthrough()
        | {"history": lambda x: history, "question": lambda x: question, "context": lambda x: retrieved_context}
        | ChatPromptTemplate.from_template(prompt_template)
        | llm
        | StrOutputParser()
    )
    result = pipeline.invoke({})
    return result

In [ ]:
qdrant_client = Qdrant()
qdrant_client.create_collection(collection_name)
qdrant_client.insert_documents(collection_name=collection_name, doc_chunks=docs, embeddings=embeddings)
question = input("Hãy nhập câu hỏi vào đây nhé: ")
historys = []
history_string = "\n".join(f"User: {chat['user']}\nBot: {chat['bot']}" for chat in historys)
while True:
    try:
        # Kiểm tra nếu người dùng muốn thoát
        if question.lower() in ["cancel", "thoát", "exit"]:
            print("Đã thoát chương trình.")
            break
        
        res = build_chain(llm, qdrant_client, collection_name=collection_name, embeddings=embeddings, question=question, history = history_string)
        print("---" * 40)
        print(f"Câu trả lời: {res}")
        question = input("Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C): ")
        historys.append({
            "user": question,
            "bot": res
        })
    except KeyboardInterrupt:
        print("\nĐã thoát chương trình bằng Ctrl+C.")
        break
    except Exception as e:
        print(f"Lỗi: {e}")
        question = input("Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C): ")

Hãy nhập câu hỏi vào đây nhé:  hi


/tmp/ipykernel_624026/302110858.py:85: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self.client.search(


------------------------------------------------------------------------------------------------------------------------------------------------------
Câu trả lời: <think>
Okay, so I'm trying to figure out how to respond to the user's message. They just said "hi," which is a common greeting. From the context provided, it looks like DetNam is an AI assistant designed to help with various tasks, especially related to camera management and data analysis using tools like Hikvision cameras, Jetson Nano, and cloud storage solutions.

First, I need to acknowledge their greeting in a friendly manner. Then, I should offer assistance based on the capabilities outlined in the provided context. The user might be looking for help with setting up camera systems, data analysis, or integrating different components like NVRs, port forwarding, or choosing appropriate databases.

I should structure my response using the specified format: [Tóm tắt] - [Giải thích]- [Ví dụ mẫu]- [Flow chart] - [Kết luận]. H

Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  loại camera nào được sử dụng trong ai camera


/tmp/ipykernel_624026/302110858.py:85: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self.client.search(


...

/tmp/ipykernel_624026/302110858.py:85: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self.client.search(


------------------------------------------------------------------------------------------------------------------------------------------------------
Câu trả lời: <think>
Okay, so I'm trying to figure out how to set up port forwarding for remote access to an NVR using Hikvision cameras. I remember that port forwarding is necessary when you want to access your NVR from outside your local network. But I'm not entirely sure about the steps involved.

First, I think I need to access my router's settings. Each router has a different way of accessing its configuration page, so I'll probably have to look up how to do that for my specific model. Usually, it involves typing an IP address like 192.168.0.1 or 192.168.1.1 into a web browser.

Once I'm in the router settings, I need to find where port forwarding is configured. I've heard that some routers have this under "Advanced Settings" or something similar. I'll look for terms like "Port Forwarding," "NAT," or "Routing."

Next, I need to know

Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  những module phân tích trên máy trạm ấy 


/tmp/ipykernel_624026/302110858.py:85: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self.client.search(


------------------------------------------------------------------------------------------------------------------------------------------------------
Câu trả lời: <think>
Okay, so I'm trying to figure out what modules are on the client machine for analysis. From the context provided, it seems like there's a focus on Jetson Nano and some software tools. Let me break this down.

First, the conversation history mentions "những module phân tích trên máy trạm ấy." So, they're asking about the analysis modules on the client machine. Looking at the retrieved context, under section 2, it talks about software and analysis tools. Specifically, for Jetson Nano, there are several modules listed: face recognition, gender and age detection, counting people, and nationality recognition.

Wait, so each of these is a module? Let me make sure I understand correctly. The user is asking about the modules on the client machine, which in this case is the Jetson Nano. So, the modules would be the different 